<a href="https://colab.research.google.com/github/iamviji/project/blob/master/Report/py_itpp_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install autoconf
!sudo apt-get install libtool
!sudo apt-get install liblapack-dev libblas-dev libfftw3-dev
!rm -rf itpp
!git clone https://github.com/iamviji/itpp.git # original one had compilation issue, did minor signle line change after forking
!ls;cd itpp;ls;mkdir -p build1 build2; cd build1 ; cmake cmake ..;make ;make install
!rm -rf py-itpp
!git clone https://github.com/iamviji/py-itpp.git
!pip3 install pybind11
!cd py-itpp;./install_prerequisites_python3.sh;make install;pip3 install -e .  
!cd py-itpp; ls ; pip3 install -e .  
!cat py-itpp/setup.py
import sys
!cp -rf py-itpp/lib/itpp py-itpp/lib/pyitpp # import itpp is not working, after this hack i started to work
sys.path.append('./py-itpp/lib')
#!pip install gwpy &> /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  automake autotools-dev libsigsegv2 m4
Suggested packages:
  autoconf-archive gnu-standards autoconf-doc libtool gettext m4-doc
The following NEW packages will be installed:
  autoconf automake autotools-dev libsigsegv2 m4
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,082 kB of archives.
After this operation, 3,994 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsigsegv2 amd64 2.12-1 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 m4 amd64 1.4.18-1 [197 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 autoconf all 2.69-11 [322 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 autotools-dev all 20180224.1 [39.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/main amd64 automake all 1:1.15.1-3ubuntu2 [509 kB]
Fet

In [1]:
import pyitpp as itpp
from matplotlib import pyplot as plt
import numpy
SNR_BEGIN = 0
SNR_END = 10
SNR_STEP_SIZE = 0.5

In [2]:
options =itpp.ivec("18 6")
NUM_OF_INPUT_MESSAGE = 1000
ind =itpp.ivec("")
ldpc_parity_reg = itpp.comm.ldpc_parity_regular (18, 3, 6, 'rand', options)
ldpc_gen_sym = itpp.comm.ldpc_gen_systematic (ldpc_parity_reg, False, ind)
ldpc_code = itpp.comm.ldpc_code (ldpc_parity_reg, ldpc_gen_sym, False)
nrof_bits = ldpc_code.get_nvar ()  - ldpc_code.get_ncheck ()
print ("nrof_bits=", nrof_bits, "nvar=", ldpc_code.get_nvar (), "ncheck=",ldpc_code.get_ncheck ())
modulator = itpp.comm.PSK(4)
output_display_counter = NUM_OF_INPUT_MESSAGE/4
bler_per_iter_ldpc_itpp_psk_4  = numpy.array(())
ber_per_iter_ldpc_itpp_psk_4  = numpy.array(())
times_per_iter_ldpc_itpp_psk_4 = numpy.array(())
for snr in numpy.arange (SNR_BEGIN, SNR_END, SNR_STEP_SIZE):
  total_bit_error = 0
  total_msg_error = 0
  total_time = 0
  total_bit = 0
  for i in range (NUM_OF_INPUT_MESSAGE):
    source_bits = itpp.base.randb(nrof_bits)
    encoded_bits = ldpc_code.encode(source_bits)
    #print ("x")
    #print (source_bits)
    #print (encoded_bits)
    tx_signal = modulator.modulate_bits(encoded_bits)
    noise_variance = 10 ** (-(snr) * 0.1)
    channel = itpp.comm.AWGN_Channel(noisevar=noise_variance)
    rx_signal = channel(tx_signal)
    #demodulated_bits = modulator.demodulate_bits(rx_signal)
    demodulated_soft_bits = modulator.demodulate_soft_bits(rx_signal, noise_variance / ldpc_code.get_rate(), itpp.comm.Soft_Method.LOGMAP)
    #print (demodulated_soft_bits)
    decoded_soft_bits = ldpc_code.decode_soft_out(demodulated_soft_bits)
    decoded_bits = ldpc_code.decode(decoded_soft_bits)
    #print (decoded_bits)
    if abs(decoded_bits != source_bits):
      total_msg_error = total_msg_error + 1
    #if (i+1) % output_display_counter == 0:
    #  total_time = timer_update(i, current_time,total_time, output_display_counter)
    total_bit_error = total_bit_error + itpp.comm.BERC.count_errors(source_bits, decoded_bits, 0, 0, 0)
    total_bit = total_bit + nrof_bits
    #print (b)
  bler = float(total_msg_error)/NUM_OF_INPUT_MESSAGE
  ber = total_bit_error / total_bit
  print('SNR: {:04.3f}:\n -> BER: {:03.2f}\n -> Total Time: {:03.2f}s'.format(snr,bler,total_time))
  ber_per_iter_ldpc_itpp_psk_4=numpy.append(ber_per_iter_ldpc_itpp_psk_4 ,ber)
  bler_per_iter_ldpc_itpp_psk_4=numpy.append(bler_per_iter_ldpc_itpp_psk_4 ,bler)
  times_per_iter_ldpc_itpp_psk_4=numpy.append(times_per_iter_ldpc_itpp_psk_4, total_time)

nrof_bits= 9 nvar= 18 ncheck= 9
SNR: 0.000:
 -> BER: 0.61
 -> Total Time: 0.00s
SNR: 0.500:
 -> BER: 0.52
 -> Total Time: 0.00s
SNR: 1.000:
 -> BER: 0.44
 -> Total Time: 0.00s
SNR: 1.500:
 -> BER: 0.32
 -> Total Time: 0.00s
SNR: 2.000:
 -> BER: 0.26
 -> Total Time: 0.00s
SNR: 2.500:
 -> BER: 0.17
 -> Total Time: 0.00s
SNR: 3.000:
 -> BER: 0.13
 -> Total Time: 0.00s
SNR: 3.500:
 -> BER: 0.07
 -> Total Time: 0.00s
SNR: 4.000:
 -> BER: 0.04
 -> Total Time: 0.00s
SNR: 4.500:
 -> BER: 0.02
 -> Total Time: 0.00s
SNR: 5.000:
 -> BER: 0.02
 -> Total Time: 0.00s
SNR: 5.500:
 -> BER: 0.01
 -> Total Time: 0.00s
SNR: 6.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 6.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 7.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 7.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 8.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 8.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 9.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 9.500:
 -> BER: 0.00
 -> Total Time: 0.00s


In [3]:
nrof_bits = 9
modulator = itpp.comm.PSK(2)
output_display_counter = NUM_OF_INPUT_MESSAGE/4
bler_per_iter_uncoded_itpp_psk_2  = numpy.array(())
#times_per_iter_uncoded_itpp_psk_2 = numpy.array(())
for snr in numpy.arange (SNR_BEGIN, SNR_END, SNR_STEP_SIZE):
  total_bit_error = 0
  total_msg_error = 0
  total_time = 0
  #current_time = time.time()
  for i in range (NUM_OF_INPUT_MESSAGE):
    source_bits = itpp.base.randb(nrof_bits)
    tx_signal = modulator.modulate_bits(source_bits)
    noise_variance = 10 ** (-(snr) * 0.1)
    channel = itpp.comm.AWGN_Channel(noisevar=noise_variance)
    rx_signal = channel(tx_signal)
    demodulated_bits = modulator.demodulate_bits(rx_signal)
    if abs(demodulated_bits != source_bits):
      total_msg_error = total_msg_error + 1
    #if (i+1) % output_display_counter == 0:
    #  total_time = timer_update(i, current_time,total_time, output_display_counter)
    #b = itpp.comm.BERC.count_errors(source_bits, demodulated_bits, 0, 0, 0)
    #print (b)
  ber = float(total_msg_error)/NUM_OF_INPUT_MESSAGE
  print('SNR: {:04.3f}:\n -> BER: {:03.2f}\n -> Total Time: {:03.2f}s'.format(snr,ber,total_time))
  bler_per_iter_uncoded_itpp_psk_2=numpy.append(bler_per_iter_uncoded_itpp_psk_2 ,ber)
  #times_per_iter_uncoded_itpp_psk_2=numpy.append(times_per_iter_uncoded_itpp_psk_2, total_time)

SNR: 0.000:
 -> BER: 0.51
 -> Total Time: 0.00s
SNR: 0.500:
 -> BER: 0.47
 -> Total Time: 0.00s
SNR: 1.000:
 -> BER: 0.41
 -> Total Time: 0.00s
SNR: 1.500:
 -> BER: 0.39
 -> Total Time: 0.00s
SNR: 2.000:
 -> BER: 0.27
 -> Total Time: 0.00s
SNR: 2.500:
 -> BER: 0.21
 -> Total Time: 0.00s
SNR: 3.000:
 -> BER: 0.22
 -> Total Time: 0.00s
SNR: 3.500:
 -> BER: 0.14
 -> Total Time: 0.00s
SNR: 4.000:
 -> BER: 0.10
 -> Total Time: 0.00s
SNR: 4.500:
 -> BER: 0.07
 -> Total Time: 0.00s
SNR: 5.000:
 -> BER: 0.06
 -> Total Time: 0.00s
SNR: 5.500:
 -> BER: 0.04
 -> Total Time: 0.00s
SNR: 6.000:
 -> BER: 0.02
 -> Total Time: 0.00s
SNR: 6.500:
 -> BER: 0.01
 -> Total Time: 0.00s
SNR: 7.000:
 -> BER: 0.01
 -> Total Time: 0.00s
SNR: 7.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 8.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 8.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 9.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 9.500:
 -> BER: 0.00
 -> Total Time: 0.00s


In [4]:
nrof_bits = 10
modulator = itpp.comm.PSK(2)
output_display_counter = NUM_OF_INPUT_MESSAGE/4
bler_per_iter_uncoded_10_itpp_psk_2  = numpy.array(())
#times_per_iter_uncoded_itpp_psk_2 = numpy.array(())
for snr in numpy.arange (SNR_BEGIN, SNR_END, SNR_STEP_SIZE):
  total_bit_error = 0
  total_msg_error = 0
  total_time = 0
  #current_time = time.time()
  for i in range (NUM_OF_INPUT_MESSAGE):
    source_bits = itpp.base.randb(nrof_bits)
    tx_signal = modulator.modulate_bits(source_bits)
    noise_variance = 10 ** (-(snr) * 0.1)
    channel = itpp.comm.AWGN_Channel(noisevar=noise_variance)
    rx_signal = channel(tx_signal)
    demodulated_bits = modulator.demodulate_bits(rx_signal)
    if abs(demodulated_bits != source_bits):
      total_msg_error = total_msg_error + 1
    #if (i+1) % output_display_counter == 0:
    #  total_time = timer_update(i, current_time,total_time, output_display_counter)
    #b = itpp.comm.BERC.count_errors(source_bits, demodulated_bits, 0, 0, 0)
    #print (b)
  ber = float(total_msg_error)/NUM_OF_INPUT_MESSAGE
  print('SNR: {:04.3f}:\n -> BER: {:03.2f}\n -> Total Time: {:03.2f}s'.format(snr,ber,total_time))
  bler_per_iter_uncoded_10_itpp_psk_2=numpy.append(bler_per_iter_uncoded_10_itpp_psk_2 ,ber)
  

SNR: 0.000:
 -> BER: 0.58
 -> Total Time: 0.00s
SNR: 0.500:
 -> BER: 0.53
 -> Total Time: 0.00s
SNR: 1.000:
 -> BER: 0.47
 -> Total Time: 0.00s
SNR: 1.500:
 -> BER: 0.39
 -> Total Time: 0.00s
SNR: 2.000:
 -> BER: 0.32
 -> Total Time: 0.00s
SNR: 2.500:
 -> BER: 0.24
 -> Total Time: 0.00s
SNR: 3.000:
 -> BER: 0.20
 -> Total Time: 0.00s
SNR: 3.500:
 -> BER: 0.15
 -> Total Time: 0.00s
SNR: 4.000:
 -> BER: 0.12
 -> Total Time: 0.00s
SNR: 4.500:
 -> BER: 0.08
 -> Total Time: 0.00s
SNR: 5.000:
 -> BER: 0.06
 -> Total Time: 0.00s
SNR: 5.500:
 -> BER: 0.03
 -> Total Time: 0.00s
SNR: 6.000:
 -> BER: 0.02
 -> Total Time: 0.00s
SNR: 6.500:
 -> BER: 0.01
 -> Total Time: 0.00s
SNR: 7.000:
 -> BER: 0.01
 -> Total Time: 0.00s
SNR: 7.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 8.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 8.500:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 9.000:
 -> BER: 0.00
 -> Total Time: 0.00s
SNR: 9.500:
 -> BER: 0.00
 -> Total Time: 0.00s


In [ ]:
options =itpp.ivec("18 6")
NUM_OF_INPUT_MESSAGE = 1000
ind =itpp.ivec("")
ldpc_parity_reg = itpp.comm.ldpc_parity_regular (20, 5, 10, 'rand', options)
ldpc_gen_sym = itpp.comm.ldpc_gen_systematic (ldpc_parity_reg, False, ind)
ldpc_code = itpp.comm.ldpc_code (ldpc_parity_reg, ldpc_gen_sym, False)
nrof_bits = ldpc_code.get_nvar ()  - ldpc_code.get_ncheck ()
print ("nrof_bits=", nrof_bits, "nvar=", ldpc_code.get_nvar (), "ncheck=",ldpc_code.get_ncheck ())
modulator = itpp.comm.PSK(4)
output_display_counter = NUM_OF_INPUT_MESSAGE/4
bler_per_iter_ldpc_10_5_10_itpp_psk_4  = numpy.array(())
ber_per_iter_ldpc_10_5_10_itpp_psk_4  = numpy.array(())

for snr in numpy.arange (SNR_BEGIN, SNR_END, SNR_STEP_SIZE):
  total_bit_error = 0
  total_msg_error = 0
  total_time = 0
  total_bit = 0
  for i in range (NUM_OF_INPUT_MESSAGE):
    source_bits = itpp.base.randb(nrof_bits)
    encoded_bits = ldpc_code.encode(source_bits)
    #print ("x")
    #print (source_bits)
    #print (encoded_bits)
    tx_signal = modulator.modulate_bits(encoded_bits)
    noise_variance = 10 ** (-(snr) * 0.1)
    channel = itpp.comm.AWGN_Channel(noisevar=noise_variance)
    rx_signal = channel(tx_signal)
    #demodulated_bits = modulator.demodulate_bits(rx_signal)
    demodulated_soft_bits = modulator.demodulate_soft_bits(rx_signal, noise_variance / ldpc_code.get_rate(), itpp.comm.Soft_Method.LOGMAP)
    #print (demodulated_soft_bits)
    decoded_soft_bits = ldpc_code.decode_soft_out(demodulated_soft_bits)
    decoded_bits = ldpc_code.decode(decoded_soft_bits)
    #print (decoded_bits)
    if abs(decoded_bits != source_bits):
      total_msg_error = total_msg_error + 1
    #if (i+1) % output_display_counter == 0:
    #  total_time = timer_update(i, current_time,total_time, output_display_counter)
    total_bit_error = total_bit_error + itpp.comm.BERC.count_errors(source_bits, decoded_bits, 0, 0, 0)
    total_bit = total_bit + nrof_bits
    #print (b)
  bler = float(total_msg_error)/NUM_OF_INPUT_MESSAGE
  ber = total_bit_error / total_bit
  print('SNR: {:04.3f}:\n -> BER: {:03.2f}\n -> Total Time: {:03.2f}s'.format(snr,bler,total_time))
  ber_per_iter_ldpc_10_5_10_itpp_psk_4=numpy.append(ber_per_iter_ldpc_10_5_10_itpp_psk_4 ,ber)
  bler_per_iter_ldpc_10_5_10_itpp_psk_4=numpy.append(bler_per_iter_ldpc_10_5_10_itpp_psk_4 ,bler)


nrof_bits= 10 nvar= 20 ncheck= 10
SNR: 0.000:
 -> BER: 0.76
 -> Total Time: 0.00s
SNR: 0.500:
 -> BER: 0.71
 -> Total Time: 0.00s
SNR: 1.000:
 -> BER: 0.60
 -> Total Time: 0.00s
SNR: 1.500:
 -> BER: 0.54
 -> Total Time: 0.00s


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
snrs = numpy.arange (SNR_BEGIN, SNR_END, SNR_STEP_SIZE)
fig, (ax1) = plt.subplots(1,1,figsize=(8,6))
ax1.semilogy(snrs,bler_per_iter_ldpc_itpp_psk_4,'', label="bler_per_iter_ldpc_itpp_psk_4")
ax1.semilogy(snrs,bler_per_iter_uncoded_10_itpp_psk_2,'', label="bler_per_iter_uncoded_10_itpp_psk_2")
ax1.semilogy(snrs,bler_per_iter_uncoded_itpp_psk_2,'', label="bler_per_iter_uncoded_10_itpp_psk_2")
ax1.semilogy(snrs,bler_per_iter_ldpc_10_5_10_itpp_psk_4,'', label="bler_per_iter_ldpc_10_5_10_itpp_psk_4")

ax1.set_ylabel('BLER')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()